In [2]:
GROUP = 0

In [6]:
from pathlib import Path
import re
import os
import glob
from pathlib import Path
import json
from shutil import copyfile
import itertools
from collections import Counter
import pandas as pd
import numpy as np

import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
from IPython.display import display

import spacy

from spacy import displacy
from spacy.pipeline import EntityRuler
from spacy.matcher import PhraseMatcher
from spacy.tokens import Doc, Span, Token
from spacy.attrs import intify_attrs

def display_ent(doc):
    displacy.render(doc, style="ent", jupyter=True)
        
base_path = Path('..')
events_path = base_path / 'events'
dictionary_path = base_path / 'dictionary'
patterns_path = dictionary_path / 'patterns'
group_events_path = events_path / f'group_{GROUP}_events.csv'
labelled_path = events_path / f'group_{GROUP}_labelled.csv'
processed_path = events_path / f'group_{GROUP}_processed.csv'

In [8]:
df = pd.read_csv(processed_path)
df.head()

,event_id,event_text_vector,STRAT_ ashburton formation,STRAT_ brockman iron formation,STRAT_ capricorn formation,STRAT_ cawse monzogranite,STRAT_ corboy formation,STRAT_ dales gorge member,STRAT_ duck creek dolomite,STRAT_ emull gabbro,...,TIMESCALE_archaean,TIMESCALE_cainozoic,TIMESCALE_devonian,TIMESCALE_mesoproterozoic,TIMESCALE_mesozoic,TIMESCALE_phanerozoic,TIMESCALE_proterozoic,TIMESCALE_tertiary,TIMESCALE_unknown,label
0,a080918_e9_1443_annual_09_13904956_0,[-0.65880835 2.0076575 -2.3191159 3.762255...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,a080918_e9_1443_annual_09_13904956_15,[-1.0317131 2.780362 2.1162007 1.628280...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,a080918_e9_1443_annual_09_13904956_18,[-3.6325414 6.233181 -0.20545405 4.883079...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,a080918_e9_1443_annual_09_13904956_21,[-1.6911434 4.5786643 -0.6014996 5.477430...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,a080918_e9_1443_annual_09_13904956_34,[-0.5821011 2.6584122 -0.9980737 -1.459626...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1


In [9]:
from sklearn.model_selection import train_test_split

X = df.drop(columns='label')
y = df['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(227, 659) (57, 659) (227,) (57,)


In [11]:
from lazypredict.Supervised import LazyClassifier

clf = LazyClassifier(verbose=0, ignore_warnings=True, custom_metric=None)
models, predictions = clf.fit(X_train, X_test, y_train, y_test)
models

/Users/rossgreen/.ve/loudml/lib/python3.7/site-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.utils.testing module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.utils. Anything that cannot be imported from sklearn.utils is now part of the private API.
  warnings.warn(message, FutureWarning)
 97%|█████████▋| 30/31 [00:03<00:00,  7.09it/s]

Learning rate set to 0.00547
0:	learn: 0.6926386	total: 59.4ms	remaining: 59.4s
1:	learn: 0.6920576	total: 63.7ms	remaining: 31.8s
2:	learn: 0.6916255	total: 67.7ms	remaining: 22.5s
3:	learn: 0.6909456	total: 71.8ms	remaining: 17.9s
4:	learn: 0.6905897	total: 75.8ms	remaining: 15.1s
5:	learn: 0.6899879	total: 79.6ms	remaining: 13.2s
6:	learn: 0.6895648	total: 83.7ms	remaining: 11.9s
7:	learn: 0.6891668	total: 88ms	remaining: 10.9s
8:	learn: 0.6886710	total: 92.1ms	remaining: 10.1s
9:	learn: 0.6881939	total: 96.2ms	remaining: 9.53s
10:	learn: 0.6877214	total: 100ms	remaining: 9.02s
11:	learn: 0.6871528	total: 104ms	remaining: 8.59s
12:	learn: 0.6866492	total: 108ms	remaining: 8.21s
13:	learn: 0.6861370	total: 112ms	remaining: 7.89s
14:	learn: 0.6858481	total: 114ms	remaining: 7.51s
15:	learn: 0.6852711	total: 119ms	remaining: 7.29s
16:	learn: 0.6848867	total: 123ms	remaining: 7.09s
17:	learn: 0.6845732	total: 127ms	remaining: 6.9s
18:	learn: 0.6840930	total: 131ms	remaining: 6.75s
19:	l

100%|██████████| 31/31 [00:08<00:00,  3.70it/s]

988:	learn: 0.5245739	total: 4.01s	remaining: 44.6ms
989:	learn: 0.5244757	total: 4.01s	remaining: 40.5ms
990:	learn: 0.5243537	total: 4.02s	remaining: 36.5ms
991:	learn: 0.5242433	total: 4.02s	remaining: 32.5ms
992:	learn: 0.5241559	total: 4.03s	remaining: 28.4ms
993:	learn: 0.5240445	total: 4.03s	remaining: 24.4ms
994:	learn: 0.5239562	total: 4.04s	remaining: 20.3ms
995:	learn: 0.5238237	total: 4.04s	remaining: 16.2ms
996:	learn: 0.5237128	total: 4.05s	remaining: 12.2ms
997:	learn: 0.5236145	total: 4.06s	remaining: 8.13ms
998:	learn: 0.5234759	total: 4.06s	remaining: 4.07ms
999:	learn: 0.5233878	total: 4.07s	remaining: 0us


,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
LinearSVC,0.65,0.63,0.63,0.64,0.22
RidgeClassifierCV,0.61,0.59,0.59,0.61,0.06
RidgeClassifier,0.61,0.59,0.59,0.61,0.05
PassiveAggressiveClassifier,0.60,0.58,0.58,0.59,0.07
KNeighborsClassifier,0.60,0.56,0.56,0.57,0.07
SGDClassifier,0.58,0.55,0.55,0.56,0.06
NuSVC,0.60,0.54,0.54,0.54,0.14
BaggingClassifier,0.61,0.54,0.54,0.50,0.15
CatBoostClassifier,0.61,0.54,0.54,0.50,4.54
